# Constructing helper functions

In [4]:
import numpy as np

In [3]:
monoStableStates ={'X1':'H', 'X2':'L'} # single dictionary
hSet = set()
lSet = set()
for node in monoStableStates:
    if monoStableStates[node] == 'H':
        hSet.add(node)
    else:
        lSet.add(node)

In [40]:
# state of in edges
def stateInEdges(inStates, nodeStates):
    inEdges = []
    for node in inStates:
        if node[1] ==  nodeState:
            inEdges.append(str(node[0]))
        else:
            inEdges.append('~'+str(node[0]))
    return inEdges

# logic combine of inedges
def combineInEdges(inEdges, logic):
    binLogic = np.log2(logic)
    binLogic = [int(x) for x in binLogic]
    start = 0
    currStringList = []
    for i in range(len(binLogic)):
        currentBin = binLogic[i]
        tempString = '+'.join(inEdges[start:start+currentBin])
        tempString = '('+tempString+')'
        currStringList.append(tempString)
        start += currentBin
    
    return ''.join(currStringList)
    
# inStates variable is in order of inLogic for one node
def parseInEdge(inStates, inLogic, nodeState, nodeName):
    variables = []
    
    # set edges type
    inEdges = stateInEdges(inStates,nodeState)
    
    
    #combine to logic
    inEdgeString = combineInEdges(inEdges, inLogic)
    
    return str(nodeName) +':' + inEdgeString
    

In [41]:
nodeStates = {'SOX17':'H', 'PRDMI':'H', 'HAND1':'L', 'ID1':'H','TAFP2C':'H'}

In [42]:
inStates = []
for key in nodeStates:
    inStates.append([key,nodeStates[key]])

In [43]:
inLogic = [4,4]

In [44]:
nodeName = 'HAND1'
nodeState = nodeStates[nodeName]

In [45]:
parseInEdge(inStates,inLogic, nodeState, nodeName)

'HAND1:(~SOX17+~PRDMI)(HAND1+~ID1)'

For each node the key input is nodeList (in order) and the the inLogic.

NodeList in order can have many choices for the same state of nodes.

So restrictions (from biology) should be added to when dealing with GRN.

# Construct the probability database based on sampling for each logic

* For monostability

Need two dictionaries such that $P(\theta_{\max} < p_i)$ or $P( p_i  <\theta_{\min} )$

* For bistability 

 The three dictionaries we would like to construct from all sample of total order is 

1. $P(\theta_{\min} < p_i < p_j)$: proxy $P(\theta_0 < p_i < p_j)$

2. $P( p_i < p_j < \theta_{\max})$: proxy $P( p_i < p_j  < \theta_0 )$

3. $P( p_i < \theta_{\min} \text{ and } p_j < \theta_{\max})$: proxy $P( p_i < \theta_0 \text{ and } p_j  < \theta_1 )$

These three dictionaries is  logic independent  should be regarded a three big whole dictionaries( but in practice can be stored to save memory)

* For multi stability

Need new database such that $\textbf{Prob}(\wedge_{\text{multiple stable states}} \{p_i < \theta_0 \text{ or } p_j > \theta_0 \})$

* NOR Gate (four stable states)

Like above

* Unknown states or node (with multiple partial stable states)

Need new database such that $\textbf{Prob}(\vee_{\text{all possible states for unknown node}}\{\wedge_{\text{multiple stable states}}  \{p_i < \theta_0 \text{ or } p_j > \theta_0 \})$. The union can be represented by the longest interval of $(p_i,p_j)$ for choosing the unknown states.

We are going to construct three distance dictionareis to store all the values we need in auto design path choice algorithm

The dictionary has 

$\textbf{Key}$: $(n_d, n_s, s)$, where $n_d$ is the number of corresponding variables that changes the state, where $n_s$ is the number of edges that does not change the state, s is u or l that indicates the states of $n_s$ variables.

$\textbf{Value}$: list of pairs ((p_i,p_j), probability), probability is one of the probability in the previous discussion and the $p_i,p_j$ has $n_d$ different variables at corresponding position, $n_s$ same variables in the rest positions with value $u,l$ indicated by $s$. It should be sorted from highest to loweset with respect to probability term.

When $p_i,p_j$ is known we can easily generate nodeList (in order) and inLogic. Note, it is easy to see the nodeList may have many choices by local symmetry

A proxy probability or ranking is $\textbf{Prob}(p_i < \theta_0 \text{ and } p_j > \theta_0, i\in I ,j\in J)$. The $\theta_{\min},\theta_{\max}$ version is an increasing function of the $\theta_0$ version, so it can be a reasonble proxity of ranking in choosing input edges type